# TinyLlama 1.1B Chat: From GGUF to Interactive Inference

Learn how to load the TinyLlama‑1.1B‑Chat‑v1.0 GGUF model from Hugging Face, run fast inference, customize prompts, and deploy a lightweight chat API. This lesson covers environment setup, model loading, generation, batching, and basic fine‑tuning using the Hugging Face ecosystem.

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Install and configure the required Python packages for GGUF inference.
2. Load the TinyLlama‑1.1B‑Chat model and tokenizer from the Hugging Face Hub.
3. Generate text with custom prompts and control generation parameters.
4. Deploy the model as a simple REST API for real‑time chat.


## Prerequisites

- Basic Python programming (3.8+).
- Familiarity with Hugging Face Transformers API.


## Setup

Let's install the required packages and set up our environment.


In [ ]:
# Create requirements.txt
requirements = '''transformers==4.35.0
torch==2.0.1
accelerate==0.21.0
huggingface_hub==0.20.3
fastapi==0.110.0
uvicorn==0.29.0'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print('✅ requirements.txt created!')

## Step 1: Environment & Dependencies

Welcome to the first step of your TinyLlama adventure! Think of this section as setting up a tiny kitchen before you start cooking a fancy dish. We’ll install the right tools (Python packages), make sure your computer can talk to the Hugging Face Hub, and set up a safe place to keep your credentials.

**Why do we need all this?**
- `transformers` gives us the recipe for TinyLlama.
- `torch` is the engine that runs the math.
- `accelerate` helps us use your GPU (if you have one) or CPU efficiently.
- `huggingface_hub` lets us download the GGUF file.
- `fastapi` and `uvicorn` are for the future chat API.

We’ll also show you how to create a virtual environment so your project stays tidy.


In [ ]:
# 1️⃣ Create a virtual environment (recommended)
# This keeps your project dependencies isolated.
# If you already have one, skip to the pip install step.

import subprocess, sys

try:
    subprocess.check_call([sys.executable, "-m", "venv", "venv"])
    print("✅ Virtual environment 'venv' created.")
except Exception as e:
    print("⚠️  Virtual environment creation failed:", e)

# 2️⃣ Activate the environment (Unix/macOS)
#    source venv/bin/activate
#    (Windows) venv\Scripts\activate.bat

# 3️⃣ Install the required packages
# We pin exact versions for reproducibility.

requirements = [
    "transformers==4.35.0",
    "torch==2.0.1",
    "accelerate==0.21.0",
    "huggingface_hub==0.20.3",
    "fastapi==0.110.0",
    "uvicorn==0.29.0"
]

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", *requirements])
    print("✅ All packages installed successfully.")
except subprocess.CalledProcessError as e:
    print("⚠️  Package installation failed. Try running pip manually.")
    sys.exit(1)


## 4️⃣ Set up your Hugging Face token

If you’re downloading a private model, you’ll need a **HF_TOKEN**. For public models like TinyLlama‑1.1B‑Chat‑v1.0, it’s optional, but it speeds up downloads.

**How to get one:**
1. Sign in to [Hugging Face](https://huggingface.co/).
2. Go to *Settings → Access Tokens*.
3. Click *New token*, give it a name, and copy the value.

**Why store it in an environment variable?**
- Keeps it out of your code.
- Lets `huggingface_hub` automatically pick it up.


In [ ]:
# Export the token for Unix/macOS
# Replace YOUR_HF_TOKEN with the string you copied.
# You can add this line to ~/.bashrc or ~/.zshrc for persistence.

import os

os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"  # <-- replace this
print("✅ HF_TOKEN set in environment (for this session).")

# For Windows PowerShell:
#   $env:HF_TOKEN = "YOUR_HF_TOKEN"


## 🎉 You’re all set!

Run the cells above in a Jupyter notebook or a Python script. If everything prints `✅`, you’re ready to load TinyLlama in the next section. If you hit any errors, double‑check:
- Python version (`python --version` should be 3.8+).
- Internet connection.
- Correct spelling of package names.

Happy coding! 🚀

## Step 2: Understanding GGUF and TinyLlama

Before we can load TinyLlama, let’s demystify two key concepts:

1. **GGUF** – Think of a GGUF file as a *tiny, super‑compressed cookbook*. It stores the model’s weights in a binary format that’s smaller than the usual `.bin` or `.pt` files, but still contains all the information the model needs to talk.
2. **TinyLlama‑1.1B‑Chat** – This is a lightweight version of the Llama family, trimmed down to 1.1 billion parameters. It’s like a compact smartphone that still runs most apps, but uses less memory and CPU.

Why GGUF? The main benefits are:
- **Speed** – Loading is faster because the file is smaller.
- **Memory** – Less RAM is required to keep the model in memory.
- **Portability** – You can ship the model to edge devices or cloud functions with minimal bandwidth.

Below we’ll inspect the GGUF file, check its size, and confirm that the Hugging Face Hub can handle it.


In [ ]:
# 1️⃣ Inspect the GGUF file size and metadata
# We’ll use the `huggingface_hub` library to download the file locally
# and then read a few bytes to confirm it’s a GGUF file.

import os
from huggingface_hub import hf_hub_download

# Path where the GGUF will be stored
model_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
filename = "TinyLlama-1.1B-Chat-v1.0.Q4_K_M.gguf"  # one of the quantized variants

# Download (or use cached copy)
local_path = hf_hub_download(repo_id=model_id, filename=filename)
print(f"✅ GGUF file downloaded to: {local_path}")

# Show file size in megabytes
size_mb = os.path.getsize(local_path) / (1024 ** 2)
print(f"📦 File size: {size_mb:.2f} MB")

# Peek at the first few bytes – GGUF files start with the ASCII string 'GGUF'
with open(local_path, "rb") as f:
    header = f.read(4)
print(f"🗂️ Header bytes: {header}")


### What we just did
- **`hf_hub_download`** pulls the file from Hugging Face and caches it locally.
- We printed the file size to see how lightweight it is.
- The header check confirms the file is indeed a GGUF binary.

If you see `b'GGUF'` in the header, you’re good to go!


In [ ]:
# 2️⃣ Quick sanity check: load the model with transformers
# This will load the GGUF file into memory. It may take a few seconds.
# We’ll use the `trust_remote_code=True` flag to allow the model class to be loaded.

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load tokenizer – TinyLlama uses the Llama tokenizer
print("🔄 Loading tokenizer…")
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
# The tokenizer is the same as the original Llama model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print("✅ Tokenizer loaded.")

# Load the GGUF model
print("🔄 Loading GGUF model…")
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
print("✅ GGUF model loaded.")


### TL;DR
- GGUF is a compact, binary format for model weights.
- TinyLlama‑1.1B‑Chat is a 1.1 billion‑parameter Llama variant, great for quick inference.
- The Hugging Face Hub and `transformers` make it trivial to download and load these files.

In the next section we’ll actually generate text with this model.


## Step 3: Loading the Model & Tokenizer

Now that we’ve downloaded the TinyLlama GGUF file, it’s time to bring it into memory. Think of the model as a *recipe book* and the tokenizer as the *chef’s measuring spoon*. The tokenizer turns your words into numbers the model can understand, and the model uses those numbers to predict the next word.

We’ll load both components with a single line each, using the `transformers` library. The `trust_remote_code=True` flag tells Hugging Face that it’s okay to run the custom code that ships with TinyLlama.

If you’re on a machine with a GPU, the model will automatically use it. If not, it will fall back to the CPU.

Let’s get started!


In [ ]:
# 1️⃣ Load the tokenizer
# The tokenizer is the same as the original Llama tokenizer
# It converts text to token ids and back.

from transformers import AutoTokenizer

model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"

try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
        use_fast=True  # fast tokenizers are a bit faster
    )
    print("✅ Tokenizer loaded successfully.")
except Exception as e:
    print("⚠️  Failed to load tokenizer:", e)
    raise

# Quick sanity check: encode a simple sentence
sample = "Hello, TinyLlama!"
encoded = tokenizer.encode(sample, add_special_tokens=True)
print("🔢 Token IDs:", encoded)
print("🧩 Decoded back:", tokenizer.decode(encoded))


### What just happened?
- `AutoTokenizer.from_pretrained` pulls the tokenizer files from the Hugging Face Hub.
- `add_special_tokens=True` adds the beginning‑of‑sentence token that TinyLlama expects.
- The quick encode/decode round‑trip confirms the tokenizer works.

Now we’re ready to load the heavy‑lifting part: the model itself.


In [ ]:
# 2️⃣ Load the GGUF model
# This step can take a few seconds, especially on a CPU.
# If you have a GPU, the model will automatically use it.

from transformers import AutoModelForCausalLM
import torch

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto",  # let accelerate decide where to place layers
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    print("✅ GGUF model loaded successfully.")
except Exception as e:
    print("⚠️  Failed to load model:", e)
    raise

# Verify that the model can run a quick forward pass
input_ids = tokenizer.encode("Tell me a joke.", return_tensors="pt")
if torch.cuda.is_available():
    input_ids = input_ids.to("cuda")

with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=20)

print("🤖 Model output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


### Why `device_map="auto"`?
- On a GPU, the model’s layers are split across memory to avoid overflow.
- On a CPU, it keeps everything in RAM but still uses the most efficient layout.

If you want to force the model onto a specific device, you can replace `device_map="auto"` with `device_map="cpu"` or `device_map="cuda:0"`.


## Knowledge Check

Test your understanding with these questions:


### Question 1

Which library is required to load GGUF models in Hugging Face Transformers?

A. torch
B. transformers
C. accelerate
D. huggingface_hub

**Answer:** B

**Explanation:** The `transformers` library provides the `AutoModelForCausalLM.from_pretrained` method that supports GGUF format via the `trust_remote_code=True` flag.
